# Chroma 支持说明与演示
本笔记展示了本工程中对 Chroma 检索的支持，包括接口调用与本地集合文件结构的说明。


## 修改概览
- 后端 main.py 的 `/search` 接口增加了 `provider` 与 `save_results` 字段，并将参数传递给搜索服务。
- 后端 services/search_service.py 的 `SearchService.search` 新增 Chroma 文件检索分支，会优先检测 `backend/03-vector-store/chroma/<collection_id>.json`。若存在则：
  - 读取其中的 `embedding_provider` 与 `embedding_model`，使用同配置生成查询向量；
  - 对每条记录的向量进行余弦相似度计算，结合 `top_k`、`threshold` 和 `word_count_threshold` 过滤并排序；
  - 如启用 `save_results`，把检索结果保存到 `backend/04-search-results/`。
- 前端 frontend/src/pages/Search.jsx 的 `handleSearch` 请求体增加了 `provider: selectedProvider`，从而可以选择 chroma 发起检索。


In [ ]:
# 配置后端地址（默认使用本地 uvicorn）
import requests, json, os
BASE_URL = os.environ.get('RAG_API', 'http://127.0.0.1:8001')
print('API Base:', BASE_URL)


In [ ]:
# 查看支持的向量数据库提供商
resp = requests.get(f'{BASE_URL}/providers')
print('Providers status:', resp.status_code)
print(json.dumps(resp.json(), ensure_ascii=False, indent=2))


In [ ]:
# 列出 Chroma 的集合
resp = requests.get(f'{BASE_URL}/collections', params={'provider': 'chroma'})
print('Collections status:', resp.status_code)
collections = resp.json().get('collections', [])
print(json.dumps(collections, ensure_ascii=False, indent=2))
collection_id = collections[0]['id'] if collections else ''
print('Selected collection_id:', collection_id)


In [ ]:
# 使用 Chroma 提供商发起搜索（如集合为空，请先在前端 Indexing 页面使用 provider=chroma 建立集合）
query = '烟台银行'
payload = {
  'query': query,
  'collection_id': collection_id,
  'top_k': 3,
  'threshold': 0.3,
  'word_count_threshold': 100,
  'save_results': False,
  'provider': 'chroma'
}
print('Request payload:', json.dumps(payload, ensure_ascii=False))
resp = requests.post(f'{BASE_URL}/search', json=payload)
print('Search status:', resp.status_code)
try:
    print(json.dumps(resp.json(), ensure_ascii=False, indent=2))
except Exception as e:
    print('响应解析失败:', e, '原始响应文本:', resp.text)

In [ ]:
# 检查本地 Chroma 集合文件结构（演示 search_service 的文件检索分支）
if collection_id:
    chroma_path = os.path.join('backend', '03-vector-store', 'chroma', f'{collection_id}.json')
    print('Chroma file path:', chroma_path)
    if os.path.exists(chroma_path):
        with open(chroma_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print('Embedding provider/model:', data.get('embedding_provider'), data.get('embedding_model'))
        print('Record count:', len(data.get('records', [])))
        # 可展示一条记录的示例结构
        if data.get('records'):
            sample = data['records'][0]
            print('Sample record keys:', list(sample.keys()))
    else:
        print('未找到本地 Chroma 集合文件，可能该集合尚未用 provider=chroma 索引或文件名不匹配。')
else:
    print('未选择 collection_id。')


## 设计说明
- Chroma 查询路径通过检测 `backend/03-vector-store/chroma/<collection_id>.json` 来触发。
- 查询向量使用与集合相同的 `embedding_provider` 与 `embedding_model` 生成。
- 余弦相似度计算后按 `score` 降序排序，并支持 `top_k`、`threshold` 与 `word_count_threshold` 过滤。
- 如启用 `save_results`，会在 `backend/04-search-results/` 生成保存文件，并在响应中返回 `saved_filepath`。
- 前端已在搜索请求体中加入 `provider` 字段，因此当选择 *Chroma* 作为向量数据库时，后端能识别并走对应分支。
